Bank Marketing

In [44]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [45]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
# Imports the dataset in a parquet format.
df_reviews = spark.read.parquet("file:///home/hduser/Downloads/bank_marketing.parquet")

In [47]:
df_reviews.printSchema()
display(df_reviews.describe())

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: long (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



DataFrame[summary: string, age: string, job: string, marital: string, education: string, default: string, balance: string, housing: string, loan: string, contact: string, day: string, month: string, duration: string, campaign: string, pdays: string, previous: string, poutcome: string, y: string]

- The parquet format file already had a schema, therefore keeping the original dataset schema. However, if the csv were to be handled the following code would be appropriate to apply the schema, including the column names along with the data type.

Passo 1: Explore e visualize os dados brutos

In [48]:
df_reviews.show(5)
df_reviews.groupby("education").count().orderBy("count", ascending=False).show(5)

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown| single|  unknown|     no|      1|     no|  no|unknown|  5|  may|     19

+---------+-----+
|education|count|
+---------+-----+
|secondary|23202|
| tertiary|13301|
|  primary| 6851|
|  unknown| 1857|
+---------+-----+



Passo 2: Limpeza e preparação de dados

Verifique os valores ausentes e decida como tratá-los. Remova caracteres indesejados dos campos categóricos e alterne entre maiúsculas e minúsculas, quando adequadas:


In [49]:
import re

In [50]:
df_reviews = df_reviews.withColumn("job", regexp_replace(lower("job"), "[^a-z]", ""))
df_reviews = df_reviews.withColumn("marital", regexp_replace(lower("marital"), "[^a-z]", ""))
df_reviews = df_reviews.withColumn("education", regexp_replace(lower("education"), "[^a-z]", ""))

In [51]:
df_reviews.summary().show()

+-------+------------------+-------+--------+---------+-------+------------------+-------+-----+--------+-----------------+-----+-----------------+-----------------+------------------+------------------+--------+-----+
|summary|               age|    job| marital|education|default|           balance|housing| loan| contact|              day|month|         duration|         campaign|             pdays|          previous|poutcome|    y|
+-------+------------------+-------+--------+---------+-------+------------------+-------+-----+--------+-----------------+-----+-----------------+-----------------+------------------+------------------+--------+-----+
|  count|             45211|  45211|   45211|    45211|  45211|             45211|  45211|45211|   45211|            45211|45211|            45211|            45211|             45211|             45211|   45211|45211|
|   mean| 40.93621021432837|   null|    null|     null|   null|1362.2720576850766|   null| null|    null|15.80641879188693| 

-Agora que você obteve um sumário estatístico básico da tabela, o próximo passo seria selecionar as variáveis relevantes para sua análise ou modelagem. Suponhamos que você esteja interessado em gerar um modelo que prevê se um cliente irá assinar um contrato baseado em suas características demográficas e histórico de relacionamento com o banco. Neste caso, considere selecionar as seguintes variáveis: idade, emprego, estado civil, educação, padrão de pagamento, equilíbrio, moradia, empréstimo, tipo de contato, dia, mês, duração da campanha, dias desde último contato, resultado do último contato e objetivo final.

In [52]:
selected_columns = ["age", "job", "marital", "education", "default", "balance", "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome", "y"]
df_selected = df_reviews.select(selected_columns)

- Em seguida, examine as correlações entre as variáveis e também entre cada variável e a variável dependente ('y'), a fim de compreender melhor quais variáveis influenciam o comportamento do cliente em relação aos produtos bancários. Você pode calcular a matriz de correlação Pearson utilizando a biblioteca numpy:



In [58]:
from pyspark.ml.stat import Correlation

# List of columns to consider excluding 'y'
columns_without_y = [col for col in selected_columns if col != 'y']

# Calculate full pairwise correlations
corrMatrix = Correlation.corr(df_selected, columns_without_y, method="pearson").head()
pairwise_correlation_matrix = corrMatrix.correlationMatrix

# Convert to NumPy array
correlation_array = pairwise_correlation_matrix.toArray()

# Print the correlation matrix
print(correlation_array)

Py4JError: An error occurred while calling z:org.apache.spark.ml.stat.Correlation.corr. Trace:
py4j.Py4JException: Method corr([class org.apache.spark.sql.Dataset, class java.util.ArrayList, class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:750)

